# Introduction

This notebook creates a chat interface to a set of FEMA PDFs documents related to preparation for disasters and safety protocols using LangChain and GPT-4. It explores some of the issues that need to be considered for a high-risk application where safety is very important ...

- Document context
- Question Context
- Poorly represented languages

For more information see Medium Post [Researching a FEMA Disaster Bot Using LangChain and GPT-4](https://medium.com/@astrobagel/4591f26d8dcd?source=friends_link&sk=892c4ceea92f7894c88aafb136320abb).

# Setup
## Environment
This notebook runs on Python 3.9 and package versions as outlined in `environment.yml`. A miniconda environment has been supplied, which you can use with ...

1. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html) by selecting the installer that fits your OS version. Once it is installed you may have to restart your terminal (closing your terminal and opening again)
2. In this directory, open terminal
3. `conda env create -f environment.yml`
4. `conda activate stay_safe_bot`

## API Credentials

1. cp `.env.example` `.env`
2. Edit this file and set your OPEN API Key

## Data

PDF Files were downloaded from FEMA as noted in `./docs_data/fema_docs.csv`, and saved into the folder `./data`.


# Analysis

In [2]:
import os
import sys
import shutil
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader 
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
import json
import re
from googletrans import Translator
translator = Translator()

from dotenv import load_dotenv

# A little mod to enable using memory *and* getting docs. See: https://github.com/langchain-ai/langchain/issues/2256#issuecomment-1665188576
import langchain
from typing import Dict, Any, Tuple
from langchain.memory.utils import get_prompt_input_key
def _get_input_output(
    self, inputs: Dict[str, Any], outputs: Dict[str, str]
) -> Tuple[str, str]:
    if self.input_key is None:
        prompt_input_key = get_prompt_input_key(inputs, self.memory_variables)
    else:
        prompt_input_key = self.input_key
    if self.output_key is None:
        output_key = list(outputs.keys())[0]
    else:
        output_key = self.output_key
    return inputs[prompt_input_key], outputs[output_key]
  
langchain.memory.chat_memory.BaseChatMemory._get_input_output = _get_input_output

def setup_model(
    vecs_dir: str,
    docs_sublist: list,
    all_docs: list,
    prefix_file_name_to_chunks: bool = False,
    temperature: float = 0.0,
    extra_prefix: str = '',
) -> ConversationalRetrievalChain:
    
    # Subset for docs we are interested in
    docs = []
    for d in all_docs:        
        d_dict = vars(d)
        if d_dict['metadata']['source'].replace('docs_data/','') in docs_sublist:
            if len(d.page_content) > 20:
                # Add file name to content for more context
                if prefix_file_name_to_chunks:
                    file_clean = re.sub(r'docs_data\/|\.pdf', '', d_dict['metadata']['source'])  
                    file_clean = re.sub(r'\-|\_', ' ', file_clean)                
                    d.page_content = f"{extra_prefix} {file_clean}: {d.page_content}"
                docs.append(d)

    # Create vector DB directory
    if os.path.exists(vecs_dir):
        shutil.rmtree(vecs_dir)
    os.makedirs(vecs_dir)

    # Choose our models
    embedding_model = OpenAIEmbeddings()
    #chat_model = OpenAI(temperature=temperature)
    chat_model = ChatOpenAI(temperature=temperature,model_name="gpt-4")

    # Calculate embeddings
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(docs, embedding=embedding_model,persist_directory=vecs_dir)
    vectordb.persist()
    
    # Set up chat
    memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', output_key='answer', return_messages=True)
    pdf_qa = ConversationalRetrievalChain.from_llm(chat_model, vectordb.as_retriever(), memory=memory, \
                                                   return_source_documents=True)
    return pdf_qa

def get_time_context(question: str) -> str:

    template = """Does the following question relate to 'planning' or 'taking immediate action': {question}

    Answer with one of the following: 'I am planning ahead:' or 'I need to take immediate action:' or 'ambiguous'"""

    prompt = PromptTemplate(template=template, input_variables=["question"])

    llm = OpenAI()

    llm_chain = LLMChain(prompt=prompt, llm=llm)
    answer = llm_chain.run(question)
    return answer


def ask_question(
    query: str,
    qa: object,
    output_docs: bool = True,
    preprocess_time_context: bool = False,
    auto_translate: bool = False,
) -> dict:

    # Auto-translate
    if auto_translate:
        lang = translator.detect(query)
        lang = lang.lang
        print(f"Language detected: {lang}")
        q = translator.translate(query, dest='en')
        query = q.text

    # First get the time context
    if preprocess_time_context:
        time_context = get_time_context(query)
        if 'planning' in time_context.lower():
            query = f"I am planning ahead: {query}"
        elif 'take immediate action' in time_context.lower():
            query = f"I need to take immediate action: {query}"

    print(f"\nQuestion: \n{query}")
    result = qa({"question": query})

    # Translate back to source language
    if auto_translate:
        q = translator.translate(result['answer'], dest=lang)
        result['answer'] = q.text
    
    print(f"\nAnswer:\n{result['answer']}")
    if output_docs:
        for doc in result['source_documents']:
            print('\n')
            print(json.dumps(vars(doc), indent=4))
    return result

# This will load API keys as defined in .env file
load_dotenv()

pdf_folder_path = f'./docs_data'


## Read All Our PDF Documents

In [3]:
files = os.listdir(pdf_folder_path)
files.sort()
all_docs_list = []
for file in files:
    if file.endswith(".pdf"):
        print(file)
        all_docs_list.append(file)
loader = PyPDFDirectoryLoader(pdf_folder_path)
all_docs = loader.load()

print(pdf_folder_path)
print(len(all_docs))
for d in all_docs:
    if "fema_protect-your-home_flooding.pdf" in d.metadata["source"]:
        print("\n")
        print(json.dumps(vars(d), indent=4))

cfpb_adult-fin-edyour-disaster-checklist.pdf
fema_protect-your-home_flooding.pdf
fema_protect-your-property-storm-surge.pdf
fema_protect-your-property_coastal-erosion.pdf
fema_protect-your-property_earthquakes.pdf
fema_protect-your-property_severe-wind.pdf
fema_protect-your-property_wildfire.pdf
fema_safeguard-critical-documents-and-valuables.pdf
fema_scenario_1-active_shooter-01102020.pdf
fema_scenario_10_power_outage_01102020.pdf
fema_scenario_10_power_outage_answer_key_01102020.pdf
fema_scenario_11_winter_storm_01102020.pdf
fema_scenario_11_winter_storm_answer_key_01102020.pdf
fema_scenario_12_small_business_01102020.pdf
fema_scenario_12_small_business_answer_key_01102020.pdf
fema_scenario_1_active_shooter_TTX_answer_key-01102020.pdf
fema_scenario_2-tornado_TTX_answer_key-01102020.pdf
fema_scenario_2_tornado-01102020.pdf
fema_scenario_3-wildfire_TTX_answer_key-01102020.pdf
fema_scenario_3_wildfire-01102020.pdf
fema_scenario_4-hurricane-01102020.pdf
fema_scenario_4_hurricane_flood_TT

In the above the documents have been split by page. As it happens, given the FEMA content - these being consise guides - this isn't unreasonable, but for other applications it might make sense to chunk into smaller sections using the [splitter parameter](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.pdf.PyPDFDirectoryLoader.html).

## Test using only a single document

Before we index all documents, it's useful to have a controlled test to analyze performance for a small body of content, one single document. The embedding process will split (chunk) this into parts and the LangChain pattern will still work, but being just one document we can get a better feel for performance.

In [4]:
vecs_dir = './vector_dbs/one_flood_doc'
docs = all_docs

# Subset to one document
docs_sublist = ['fema_protect-your-home_flooding.pdf']

pdf_qa = setup_model(vecs_dir, docs_sublist, docs, prefix_file_name_to_chunks=False)

## Ask 'How do I prepare my home for floods?' using one PDF document as the source

In [5]:
ask_question("How do I prepare my home for floods?", pdf_qa)


Question: 
How do I prepare my home for floods?

Answer:
There are several steps you can take to prepare your home for floods:

1. Determine the Base Flood Elevation (BFE) for your home. This is how high the water is expected to rise during flooding in high risk areas. Your local floodplain manager can help you find this information.

2. Direct water away from structures. Make sure your yard slopes away from buildings on your property and that water has a place to drain. Clear your gutters, assess drainage issues, or collect water in rain barrels.

3. Anchor fuel tanks to prevent them from tipping over or floating in a flood. 

4. Floodproof walls by adding water-resistant exterior sheathing and sealing them to prevent shallow flooding from damaging your home.

5. Secure manufactured homes to a permanent foundation to resist flotation, collapse, or side-to-side movement.

6. Document all of your belongings to help with the insurance claims process.

7. Get flood insurance as most home

{'question': 'How do I prepare my home for floods?',
 'chat_history': [HumanMessage(content='How do I prepare my home for floods?', additional_kwargs={}, example=False),
  AIMessage(content='There are several steps you can take to prepare your home for floods:\n\n1. Determine the Base Flood Elevation (BFE) for your home. This is how high the water is expected to rise during flooding in high risk areas. Your local floodplain manager can help you find this information.\n\n2. Direct water away from structures. Make sure your yard slopes away from buildings on your property and that water has a place to drain. Clear your gutters, assess drainage issues, or collect water in rain barrels.\n\n3. Anchor fuel tanks to prevent them from tipping over or floating in a flood. \n\n4. Floodproof walls by adding water-resistant exterior sheathing and sealing them to prevent shallow flooding from damaging your home.\n\n5. Secure manufactured homes to a permanent foundation to resist flotation, collapse

The answer *looks* great, and has summarized brilliantly the chunks that were returned in the retreival. However, looking at the single source document [https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-home_flooding.pdf](https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-home_flooding.pdf), notice how page 4 is missing from the retreived documents above. This PDF is a short document where *everything* in it is relevant to flood preparation, so losing pages is actually significant.

It goes to show that blindly accepting LLM patterns on the web may give amazing looking results, but work is needed to make them truly useful.

## Adding filename context to text chunks so we get all information

What about if we give a little more context to our text chunks? For example a really crude approach would be to prefix chunks with 'This snippet relates to ' and the filename, that way chunks in the above document would all have at least something to indicate they are related to flood protection. This concept could be extended to include other meta data about the documents, but let's try this crude approach ...

In [6]:
docs = all_docs
# Note the argument prefix_file_name_to_chunks=True
pdf_qa = setup_model(vecs_dir, docs_sublist, docs, prefix_file_name_to_chunks=True, extra_prefix="This snippet relates to ")
ask_question("How do I prepare my home for floods?", pdf_qa)


Question: 
How do I prepare my home for floods?

Answer:
There are several steps you can take to prepare your home for floods:

1. Direct Water Away from Structures: Make sure your yard slopes away from buildings on your property and that water has a place to drain. Clear your gutters, assess drainage issues, or collect water in rain barrels.

2. Anchor Fuel Tanks: Anchor any fuel tanks to the pad to prevent them from tipping over or floating in a flood. Spilled fuel could become a fire hazard.

3. Floodproof Walls: Add water-resistant exterior sheathing on walls and seal them to prevent shallow flooding from damaging your home. Cover openings below the Base Flood Elevation (BFE) and seal all exterior openings around pumping and equipment.

4. Secure Manufactured Homes: If you have a manufactured home, it must be affixed to a permanent foundation so that the wheels and axles do not support its weight and resist flotation, collapse, or side-to-side movement.

5. Elevate Your Home: Elev

That does now seem to have captured the important pages from this PDF, and summarized them nicely. Obviously very crude, a more formal approach using meta data instead of just the filename would be better of course, but it illustrates how context like this can be important.

## Ask 'How do I prepare my home for floods?' using ALL PDF document as the source

Now that we know we can surface appropriate content from one document, what about if we add more documents into our library?

In [7]:
vecs_dir = './vector_dbs/all_docs'
docs = all_docs
# Note the argument prefix_file_name_to_chunks=True
pdf_qa = setup_model(vecs_dir, all_docs_list, docs, prefix_file_name_to_chunks=True, extra_prefix="This snippet relates to ")
result = ask_question("How do I prepare my home for floods?", pdf_qa)



Question: 
How do I prepare my home for floods?

Answer:
There are several steps you can take to prepare your home for floods:

1. Create an emergency plan for your family and practice it regularly. When a storm is approaching, evacuate and move your car to higher ground.

2. Purchase flood insurance for your home and its contents, even if you do not live in a high-risk flood zone.

3. Document your belongings. This will help with the insurance process if you need to file a claim.

4. Store valuables and important documents above the Base Flood Elevation (BFE) in waterproof or water-resistant containers. 

5. Elevate appliances and utilities such as water heaters, washers, dryers, and electric panels on higher floors to prevent them from getting damaged by flood water.

6. Use flood-resistant materials for insulation, drywall, and floor coverings like tile to minimize damage.

7. Make sure your yard slopes away from buildings on your property and that water has a place to drain. 

8. 

That's done a great job, it surfaced our three key articles from [https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-home_flooding.pdf](https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-home_flooding.pdf), plus an aticle from [https://www.fema.gov/sites/default/files/documents/fema_protect-your-property-storm-surge.pdf](https://www.fema.gov/sites/default/files/documents/fema_protect-your-property-storm-surge.pdf), the only other document in the set which mentions flooding.

## Testing context, asking questions specific to being before/during a disaster

I have intentionally chosen a mixed set of documents where some relate to preparing for a disaster, and some for what to do during a disaster. This is to highlight how it can be important to provide some context to content, or prompts will trigger incorrect responses. Let's first try a prompt related to the timeframe *during* a disaster ...


In [82]:
result = ask_question("I live next to a canal and see the water rising right now, what should I do?", pdf_qa, output_docs=True)


Question: 
I live next to a canal and see the water rising right now, what should I do?

Answer:
If you notice water levels rising immediately, it's important to prioritize your safety. Here are some steps you can take:

1. Evacuate: If the water levels are rising rapidly, it's crucial to evacuate to higher ground as soon as possible. 

2. Move your car: If you have time, move your car to higher ground. According to the National Weather Service, just two feet of water can move a vehicle.

3. Secure your home: If you have time, secure your home by moving valuables and important documents to an upper floor or at least above the Base Flood Elevation (BFE). Place them in waterproof or water-resistant containers. 

4. Communicate: Let your family, friends, and neighbors know about the situation. If you have a pre-established meeting point, head there or inform others of your evacuation plans.

5. Contact authorities: Inform local authorities about the situation. They can provide guidance a

That's a great response, relates to what's happening right now as we'd expect. It has summarized content from three different documents. It could be argued it's very consise and misses some information, but that could be addressed with better prompting (Langchain supports a prompt template for example).

OK, what about if we are more ambiguous with our question and don't specify a timeframe ...

In [8]:
result = ask_question("I live next to a canal and see the water rising, what should I do?", pdf_qa, output_docs=False)


Question: 
I live next to a canal and see the water rising, what should I do?

Answer:
If you live next to a canal and see water levels rising, you should take the following steps:

1. Have an emergency plan: Create an emergency plan for your family and practice it regularly. This plan should include evacuation routes and a central meeting point.

2. Evacuate: If a storm is approaching and water levels are rising, it's important to evacuate to higher ground. 

3. Protect your property: Try to elevate appliances and utilities such as water heaters, washers, dryers, and electric panels to prevent them from getting damaged by flood water. You can also place sandbags in areas that are most at risk from flooding.

4. Document your belongings: Document your home’s contents visually. This will help with the insurance process if you need to file a claim.

5. Get flood insurance: Most homeowners insurance policies don’t cover flood damage. Protect your investment by purchasing flood insurance 

The answer is now a bit of a mixed bag, with some points relating to immate action "Move your car to higher ground" and some to preparation "Purchase flood insurance". It shows how having content which relates to mixed content can lead to unhelpful advice if the question doesn't provide that context. When people are stressed during an emergency, they probably aren't thinking of prompt engineering.

In [9]:
questions = [
    "I live next to a canal and the water is rising, what should I do?",
    "Help my roof is blowing off!",
    "How do I prevent my roof from blowing off in a hurricane",
    "Dog",
    "How do I prepare my home for floods?"
]
for q in questions:
    print(f"Question: {q}")
    answer = get_time_context(q)
    print(answer.strip(), "\n")


Question: I live next to a canal and the water is rising, what should I do?
I need to take immediate action: 

Question: Help my roof is blowing off!
I need to take immediate action: 

Question: How do I prevent my roof from blowing off in a hurricane
I am planning ahead. 

Question: Dog
Ambiguous 

Question: How do I prepare my home for floods?
I am planning ahead. 



Great, for these simple examples GPT-4 is able to zero-shot classify. Let's now use this as a preprocessor to our previous question ...

In [10]:
result = ask_question("I live next to a canal and see the water rising, what should I do?", pdf_qa, output_docs=False, preprocess_time_context=True)


Question: 
I need to take immediate action: I live next to a canal and see the water rising, what should I do?

Answer:
If you see the water rising next to your canal, you should immediately implement your emergency plan. This includes evacuating your home and moving your car to higher ground, as just two feet of water can move a vehicle. You should also ensure that your valuables and important documents are stored above the Base Flood Elevation (BFE), preferably on an upper floor, and placed in waterproof or water-resistant containers. If you have time, consider moving appliances and utilities such as water heaters, washers, dryers, and electric panels to higher floors to prevent them from getting damaged or ruined by flood water. Always prioritize your safety and the safety of your family over protecting property.


Perfect, it jumps straight to advice you need right now in an emergency. Let's try and example of the converse, for planning ...

In [122]:
result = ask_question("What can I do to make my house forest fire resistant?", pdf_qa, output_docs=False, preprocess_time_context=True)


Question: 
I am planning ahead: What can I do to make my house forest fire resistant?

Answer:
There are several steps you can take to make your house resistant to forest fires:

1. Install or replace your roof with a Class A-rated roof with noncombustible coverings. The roof is the most at-risk part of a house in a wildfire due to its size and orientation.

2. Install and replace exterior wall coverings with noncombustible or fire-resistant materials. A minimum fire-resistance rating of one hour for the wall assembly is recommended.

3. Create a 30 feet defensible space around your home by reducing or removing flammable vegetation and using noncombustible materials such as gravel, brick, or concrete.

4. Regularly clean and remove debris from the roof and gutters to reduce the likelihood of something catching on fire on top of your home.

5. Enclose your home's foundation to lower the chance of wind-blown embers getting underneath your home.

6. Plan for access to water by purchasing

Perfect, it captured the preparation nicely. Obviously this would need a lot of testing for anything that could be used in a real emergency, but it illustrates that care is needed with a mixed set of content.

## Making sure the model ONLY returns results for questions related to our documents

If we were to create a bot for disaster advice, the last thing we'd want is for it to start talking about how to make sponge cakes. Let's test that the Langchain pattern will do this ...

In [5]:
result = ask_question("How do I make a sponge cake?", pdf_qa, output_docs=False, preprocess_time_context=True)


Question: 
How do I make a sponge cake?

Answer:
I'm sorry, but the provided context does not contain information on how to make a sponge cake.


That's exactly what we want, responses only relate to the content we provided.

## Conversation history

Let's test conversation history, supported nicely by LangChain with minimal fuss. First let's ask a question ...

In [10]:
result = ask_question("There's a hurricane forecast, what should I do?", pdf_qa, output_docs=False, preprocess_time_context=True)


Question: 
I need to take immediate action: There's a hurricane forecast, what should I do?

Answer:
If there's a hurricane forecast, you should take the following immediate actions:

1. Check the flood risk for your area. You can do this by visiting https://msc.fema.gov/portal/home and entering your home’s address.

2. Understand the difference between a Hurricane Watch and a Hurricane Warning. A Hurricane Watch means that hurricane conditions are possible within a specified area and is issued 48 hours in advance of the anticipated onset of tropical-storm-force winds. A Hurricane Warning means that hurricane conditions are expected within a specified area and is issued 36 hours in advance of the anticipated onset of tropical-storm-force winds.

3. Take actions around your house to help reduce the impact of a flooding event. This could include placing sand bags in areas that are most at risk from flooding and elevating mechanical devices like air conditioners, generators, and circuit 

Now let's ask a followup ...

In [11]:
result = ask_question("It's about to arrive, help!", pdf_qa, output_docs=False, preprocess_time_context=True)


Question: 
It's about to arrive, help!

Answer:
Here are some immediate actions you should take for safety:

1. If you are in a flood zone, consider evacuating to a safer location. If evacuation is not possible, move to higher ground within your home.
2. Secure your home by closing and boarding up windows. Remove any outdoor items that could be picked up by the wind.
3. Gather your emergency supplies, including your Go-Kit which should include water, a first-aid kit, a flashlight, batteries, a hand-crank/solar powered radio, non-perishable food, cash, a wrench, hand sanitizer, a mylar blanket, ear plugs, and a book and/or deck of cards.
4. If you have pets, ensure you have a Go-Kit for them as well, including food, water, medication and favorite toys. Make sure they are microchipped or have ID tags.
5. Make sure your car is filled with gas and ready to go in case you need to evacuate.
6. Unplug all electrical appliances to prevent power-surge damage.
7. Stay informed about the storm's

It knew I was referring to a hurricane, so history worked nicely.

## Different languages

Though the content was in English, can we use LLMs to query in other languages?

In [8]:
q = translator.translate("How do I prepare my home for floods?", dest='pt')

print("================ RAW =================")
pdf_qa = setup_model(vecs_dir, all_docs_list, docs, prefix_file_name_to_chunks=True, extra_prefix="This snippet relates to ")
result = ask_question(q.text, pdf_qa, output_docs=True, preprocess_time_context=True)

q = translator.translate(result['answer'], dest='en')
print("\n\n=============== TRANSLATED TO ENGLISH USING GOOGLE TRANSLATE =================\n")
print(q.text)

================ RAW =================

Question: 
I am planning ahead: Como preparar minha casa para enchentes?

Answer:
Existem várias maneiras de preparar sua casa para enchentes:

1. Tenha um plano de emergência: Crie um plano de emergência para sua família e pratique-o regularmente. Quando uma tempestade se aproxima, evacue e mova seu carro para um terreno mais alto.

2. Obtenha seguro contra inundações: A maioria das apólices de seguro residencial não cobre danos causados por inundações. Proteja seu investimento comprando seguro contra inundações para sua casa e seu conteúdo.

3. Prepare ou atualize uma lista do conteúdo da sua casa: Documente seus pertences. Isso lhe dará tranquilidade e ajudará no processo de seguro se você precisar fazer uma reclamação.

4. Armazene objetos de valor: Armazene objetos de valor e documentos importantes acima do BFE (preferencialmente em um andar superior). Coloque-os em recipientes à prova d'água ou resistentes à água.

5. Eleve aparelhos e util

I asked 'How do I prepare my home for a flood?' in Portuguese, and got an answer .... in Portuguese. Asking in an entirely different, albeit well represented language seems to give great results. 

OK, what about Swahili ...

In [11]:
q = translator.translate("How do I prepare my home for floods?", dest='sw')

print("================ RAW =================")
pdf_qa = setup_model(vecs_dir, all_docs_list, docs, prefix_file_name_to_chunks=True, extra_prefix="This snippet relates to ")
result = ask_question(q.text, pdf_qa, output_docs=True, preprocess_time_context=True)

q = translator.translate(result['answer'], dest='en')
print("\n\n=============== TRANSLATED TO ENGLISH USING GOOGLE TRANSLATE =================\n")
print(q.text)

================ RAW =================

Question: 
Je, ninatayarishaje nyumba yangu kwa mafuriko?

Answer:
Kuna hatua kadhaa unazoweza kuchukua ili kulinda nyumba yako dhidi ya mafuriko:

1. Elekeza Maji Mbali na Majengo: Hakikisha eneo lako linaelekea mbali na majengo kwenye mali yako na kwamba maji yana mahali pa kumwagika. Safisha mifereji yako, tathmini masuala ya mifereji ya maji, au kukusanya maji katika mapipa ya mvua.

2. Nanga Mizinga ya Mafuta: Nanga mizinga yoyote ya mafuta kwenye pedi ili kuzuia kuzunguka au kuogelea katika mafuriko. Mafuta yaliyomwagika yanaweza kuwa hatari ya moto. Hakikisha kuwa matundu ya uingizaji hewa na ufunguzi wa mstari wa kujaza yako juu ya BFE (Base Flood Elevation). Kumbuka: Hii inaweza kuhitaji ruhusa kutoka kwa mtoa huduma wako wa mafuta.

3. Kinga Kuta dhidi ya Mafuriko: Ongeza sheathing inayostahimili maji kwenye kuta na uzibane ili kuzuia mafuriko ya kina kifupi kuharibu nyumba yako. Funika ufunguzi chini ya BFE na ziba ufunguzi wote wa nje

Asking the question in Swahili results in an answer in Swahili. However, looking at the documents returned we see some of the key flood sections are missing and we have snippets related to wildfires. The results *look* very plausible, but are missing important information, for example to insure your home. Basically, using embeddings created for English documents with Swahili questions doesn't result in good matches. Not unreasonable, though it did work for a latin language Portuguese.

A more robust approach would be to detect language and translate into English with Google translate, then using Google Translate translate the response back into Swahili. For a risk critical use-case like disaster response this would need a LOT of testing with native speakers to ensure safety.

In [22]:
# Translate to Swahili
q = translator.translate("How do I prepare my home for floods?", dest='sw')

print("================ RAW =================")
result = ask_question(q.text, pdf_qa, output_docs=False, preprocess_time_context=True, auto_translate=True)

q = translator.translate(result['answer'], dest='en')
print("\n\n=============== TRANSLATED TO ENGLISH USING GOOGLE TRANSLATE =================\n")
print(q.text)

================ RAW =================
Language detected: sw

Question: 
I am planning ahead: How do I prepare my home for flooding?

Answer:
Kuna hatua kadhaa unazoweza kuchukua ili kuandaa nyumba yako kwa mafuriko yanayoweza kutokea:

1. Tengeneza mpango wa dharura kwa ajili ya familia yako na uufanyie kazi mara kwa mara. Dhoruba inapokaribia, ondoka na usogeze gari lako hadi sehemu ya juu.

2. Nunua bima ya mafuriko kwa ajili ya nyumba yako na vilivyomo, hata kama huishi katika eneo lenye hatari kubwa ya mafuriko.

3. Andika vitu vyako. Hii itasaidia katika mchakato wa bima ikiwa unahitaji kuwasilisha dai.

4. Hifadhi vitu vya thamani na hati muhimu juu ya Mwinuko wa Mafuriko ya Msingi (BFE), ikiwezekana kwenye ghorofa ya juu, katika vyombo visivyo na maji au visivyo na maji.

5. Kuinua vifaa na huduma kama vile hita za maji, washers, vikaushio na paneli za umeme kwenye sakafu ya juu ili kuvizuia kuharibiwa na maji ya mafuriko.

6. Tumia nyenzo zinazostahimili mafuriko nyumbani kwak

## Asking about other disaster scenarios

Let's explore some other scenarios covered in the PDF documents we provided ...

In [7]:
# Set up the model again or our Swahili conversation history will be used
vecs_dir = './vector_dbs/all_docs'
docs = all_docs
pdf_qa = setup_model(vecs_dir, all_docs_list, docs, prefix_file_name_to_chunks=True, extra_prefix="This snippet relates to ")

questions = [
    "What should I do for pets if there is a forest fire?",
    "My roof is blowing off!!! Help, what should I do?!!!",
    "I live next to a brook, should I worry about floods?",
    "If I live inland, will sea level affect me?",
    "Is there a cheap way to get flood insurance?",
    "We're in an Earthquake, what should we do!!???",
    "What things should I have in an emergency kit?",
]

for q in questions:
    print("\n ===================== ")
    ask_question(q, pdf_qa, output_docs=False, preprocess_time_context=True, auto_translate=True)



Language detected: en

Question: 
I need to take immediate action: What should I do for pets if there is a forest fire?

Answer:
If there is a forest fire, you should have a Go-Kit for your pet(s) which should include food, water, medication, medical records kept in Ziploc bags, and favorite toys (if applicable). Include your veterinarian’s contact information and be sure that your pet is microchipped or at least has the proper ID tags displayed. If you need to evacuate, take your pet with you if possible. If you cannot take your pet with you, ensure they are in a safe place with access to food and water.

Language detected: en

Question: 
I need to take immediate action: My roof is blowing off!!! Help, what should I do?!!!

Answer:
The FEMA brochures do not provide specific instructions for immediate action if your roof is blowing off. However, it is generally recommended to seek immediate shelter in a safe area of your home, away from windows and potential falling debris. After the 

All seem very reasonable